# **Установка необходимых библиотек**

In [1]:
!pip install --break-system-packages clickhouse-connect

Для занятия:

Файлы лежат в каталоге /var/lib/clickhouse/data/

# **Подключаемся к базе данных**

In [4]:
import clickhouse_connect
import pandas as pd
import os

pd.options.display.max_colwidth = 100

# Вбейте свой телеграм никнейм, чтобы в случае проблем мы могли вас индефицировать
database = 'shustikov'

client = clickhouse_connect.get_client(host='clickhouse01', port=8123, username=os.getenv('CLICKHOUSE_USER'), password=os.getenv('CLICKHOUSE_PASSWORD'))

# **Создаем свое окружение**

In [ ]:
client.command(f'''
    CREATE DATABASE IF NOT EXISTS {database} ON CLUSTER '{{cluster}}';
''')

# **Типы данных**

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.type_data;
''')

client.command(f'''
    CREATE TABLE {database}.type_data
    (
    --------------------------
    -- основные типы данных --
    --------------------------
        i Int8,                               -- Int8-256 (со знаком)
        ui UInt8,                             -- UInt8-256 (без знака)
        fl Float32,                           -- Float32-64 (для мат.расчетов, но не для финансов)  
        dc Decimal(9, 4),                      -- Decimal32-256 (точность после запятой)
        st String,                            -- имеет произвольную длинну
        fst FixedString(5),                   -- имеет фиксированную длинну
    --------------------------------
    -- дополнитеьлные типы данных --
    --------------------------------
        UID UUID,                             -- уникальный идентификатор
        ip4 IPv4,                             -- 127.0.0.1
        ip6 IPv6,                             -- f2c6:e19b:da60:52ad:2cef:62fe:0279
    --------------------------------
    --    типы даты и времени     --
    --------------------------------
        dt Date,                              -- Date32 (различаются диапозном дат)
        dtm DateTime,                         -- Сохрняет время с точностью то секунд
        dtm64 DateTime64,                     -- Сохрняет время с точностью то наносекунд
    ------------------------------------
    --    композитные типы данных             -- позволяют хранить более сложные структуры данных
    ------------------------------------
        ar Array(UInt8),                      -- массив данных
        tu Tuple(Date, UInt16, Decimal32(2)), -- кортеж
        ns Nested(                            -- Вложенные структуры
            col1 String,
            col2 UInt64,
            col3 String
            ),
    mp Map(String, Int16),                    -- хранит в данные в виде ключ -> значение
    en Enum('bad' = 2,                        -- хранит данные определенного значения
            'udovlet' = 3, 
            'good' = 4 )      
    )
    ENGINE = Log;
''')


client.command(f'''
    INSERT INTO {database}.type_data
    (
               i, ui, fl, dc, st, fst,
        UID, ip4, ip6,
        dt, dtm, dtm64,
        ar, tu, 
        ns.col1, ns.col2, ns.col3,
        mp, en
    )
    VALUES 
    (
        -100,
        200,
        3.14,
        toDecimal32(3.14, 4),
        'Пример строки',
        'ABCDE',
        generateUUIDv4(),
        '192.168.1.1',
        '2001:db8::1',
        toDate('2025-04-30'),
        toDateTime('2025-04-30 14:30:00'),
        toDateTime64('2025-04-30 14:30:00.123456', 6),
        [10, 20, 30],
        (toDate('2025-04-30'), 150, 99.99),
        ['one'],
        [123456],
        ['value1'],
        {'key1': 10, 'key2': -20},
        'udovlet'
    );
''')

In [ ]:
client.query_df(f'''
    SELECT 
      *
    FROM {database}.type_data
''')

Обращение к композитрым типам данных

In [ ]:
client.query_df(f'''
    SELECT 
        ar[1],      -- обращение к эл-там массива
        ar.size0,   -- получение размера массива
        tu,         -- чтение кортежа
        ns.col1,    -- обращение к вложенной структуре
        mp['key1'], -- получение данных из Map
        en          -- Чтение Enum
    FROM {database}.type_data
''')

### Задание на самостоятельную работу


1) Необходимо содать таблицу заказов (Мы еще не проходили создание таблиц поэтому создай по примеру выше с движком `Log`)

| Поле         | Описание                            |
|--------------|-------------------------------------|
| `order_id`   | Уникальный ID заказа                |
| `user_id`    | ID пользователя                     |
| `order_date` | Дата и время оформления заказа      |
| `total_amount` | Сумма заказа                      |
| `paid`       |  Признак оплаты: оплачено, не оплачено|
| `items`      |  Список ID товаров в заказе         |

2) Тебе необходимо выбрать самому подходящие типы данных для колонок
3) После создания вставь подходящие строки через `INSERT INTO <имя БД>.<имя таблицы> VALUES (...)`
4) Выведи выборку строк

### Решение(Смотреть только после выполенения)


In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.order;
''')

client.command(f'''
    CREATE TABLE IF NOT EXISTS {database}.order
    (
        order_id     UInt64,
        user_id      UInt32,
        order_date   Date,
        total_amount Decimal(10, 2),
        paid         Enum('оплачено' = 1,
                          'не оплачено' = 0),
        items        Array(UInt32)
    )
    ENGINE = Log
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.order VALUES
        (1, 101, '2025-06-20', 2599.99, 'оплачено', [11, 42, 73]),
        (2, 102, '2025-06-22', 499.50, 'не оплачено', [5, 18]),
        (3, 101, '2025-06-24', 0, 'не оплачено', []);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM  {database}.order
''')

# **Функции к приведению типов данных**

In [ ]:
client.query_df('''
    select '1'::Int64
''')

In [ ]:
# Тут будет ошибка
client.query_df('''
    select NULL::Int64
''')

In [ ]:
client.query_df('''
    select CAST('1'  as Int8)
''')

In [ ]:
# Тут будет ошибка
client.query_df('''
   select CAST(NULL as Int8)
''')



to<Тип данных><исключение в случае ошибки приведения типа: OrNull, OrZero, OrDefault>

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.cast_type_data;
''')

client.command(f'''
CREATE TABLE {database}.cast_type_data (
    col String
)
ENGINE = Log;
''')

client.command(f'''
INSERT INTO {database}.cast_type_data values ('1'),('2'),('1a'),('-1')
''')

In [ ]:
client.query_df(f'''
    SELECT
        col,
        toInt64OrNull(col),
        toInt8OrZero(col),
        toInt8OrDefault(col, -100),
        toUInt8(-1), toUInt8(-1.1), toUInt8(256) -- выход за пределы преобразует в значение по модулю диапозона
    FROM {database}.cast_type_data
''')

### Задание на самостоятельную работу


1) Выполни запросы в следующей ячейке.
2) Сделай так чтобы UNION ALL выполнился
3) Типы данных, которые не могут быть автоматически преобразованы системой должны быть, как у таблицы 1. В случае коализий преобразования типов данных, необходимо ставить значение NULL.

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_1;
''')

client.command(f'''
CREATE TABLE  {database}.table_1
(
    order_id     UInt32, 
    user_id      UInt16,  
    total_amount Float32,   
    paid         Nullable(UInt8),     
    item_count   Int8   
)
ENGINE = Log;
''')

client.command(f'''
    INSERT INTO {database}.table_1 VALUES (10001, 321, 1499.50, 1, 3), (10001,  321, 1499.50 , NULL, 3), (10001, 321, , 1, 3);
''')

client.command(f'''
    DROP TABLE IF EXISTS {database}.table_2;
''')

client.command(f'''
CREATE TABLE  {database}.table_2
(
    order_id     Int64, 
    user_id      UInt32,  
    total_amount Nullable(Decimal(5, 2)),   
    paid         UInt8,     
    item_count   String   
)
ENGINE = Log;
''')

client.command(f'''
    INSERT INTO {database}.table_2 VALUES (10001, 123456, 899.99, 0, '5'), (10001, 123456, NULL, 0, '5'), (10001, 123456, 899.99, 0, '5 тут специально вписаны буквы') ;
''')

In [ ]:
# Исправлять ошибку
client.query_df(f'''
    SELECT order_id, user_id, total_amount, paid, item_count FROM {database}.table_1
    UNION ALL
    SELECT order_id, user_id, total_amount, paid, item_count FROM {database}.table_2
''')

### Решение(Смотреть только после выполенения)


In [ ]:
client.query_df(f'''
    SELECT order_id, user_id, total_amount, paid, item_count 
    FROM {database}.table_1
    UNION ALL
    SELECT order_id, user_id, toFloat32(total_amount), paid, toInt32OrNull(item_count)
    FROM {database}.table_2
''')

# **Создание таблиц**

Создание таблиц в ClikHouse очень похоже на создание таблиц в других базах данных, за исключением того что в КХ обязательно нужно указывать движок базы данных. Каждый движок во своему уникален, но самый встречаемый и вообще именно для него и создавался КХ это MergeTree.

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.mt;
''')

client.command(f'''
    CREATE TABLE IF NOT EXISTS {database}.mt
    (
        id UInt32,
        dt datetime
    )
    ENGINE = MergeTree                                                 -- обязательно нужно указывать движок
    PRIMARY KEY(id)                                                    -- не обязательное поле по умолчанию равно ORDER BY
    ORDER BY(id, dt)                                                   -- обязательно должны быть колонки в порядке из primary key
    TTL dt + INTERVAL 1 MONTH DELETE;                                  -- от времени dt отсчитывается 1 месяц после чего данные удаляются (данные удаляются полсе слияния)
        --dt + INTERVAL 1 MONTH DELETE WHERE toDayOfWeek(d) IN (6, 7); -- удаление с фильтрацией
        --dt + INTERVAL 1 WEEK TO VOLUME 'aaa',                        -- перемещение данных в вольюм(совокупность дисков, задаваемая в конфигах)
        --dt + INTERVAL 2 WEEK TO DISK 'bbb';                          -- перемещение данных на диск (указывается имя диска)
''')



### Самостоятельная работа на проверку работоспостобности TTL:

1) Создай таблицу с движком MergeTree c 2мя колонками: id Int32, dt DateTime
2) Укажи только сотировку по id. PRIMARY KEY указывать не нужно оно будет совпадать по умолчанию с ORDER BY
3) Напиши в условии TTL удаление строки по dt через 10 секунд
4) вставь данные через конструкцию `INSERT INTO <имя БД>.<имя таблицы> AS SELECT number, now() + number FROM numbers(100)` (`numbers(100)` это таблица которая генерирует колонку number со значениями от 0 до 99)
5) выполни select твоей таблицы и посмотри сколько в ней строк
6) как я уже говорил удаление происходит после слияния поэтому выполни команду ` OPTIMIZE TABLE <имя БД>.<имя таблицы> FINAL;` через 20 секунд после выполнения шага 5
7) выполни повторно шаг 5
8) теперь вопроряй 5-6 шаги пока в таблице совсем не останется данных

### Решение(Смотреть только после выполенения)

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_ttl;
''')

client.command(f'''
    CREATE TABLE IF NOT EXISTS {database}.table_ttl
    (
        id Int32,
        dt DateTime
    )
    ENGINE = MergeTree 
    ORDER BY (id)
    TTL dt + INTERVAL 10 SECOND DELETE;
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.mt
    SELECT 
      number,
      now() + number
    from numbers(100);
''')

In [ ]:
client.command(f'''
    OPTIMIZE TABLE {database}.mt FINAL;
''')

# **Описание полей при создании таблиц**


In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.test_fields_without_ttl;
''')


client.command(f'''
    CREATE TABLE {database}.test_fields_without_ttl
    (
          col_default UInt64 DEFAULT 42                         -- значение по умолчанию
        , col_materialized UInt64 MATERIALIZED col_default * 33 -- к данной колонке можно обратиться только по имени
        , col_alias UInt64 ALIAS col_default + 1                -- к данной колонке можно обратиться только по имени
        , col_codec String CODEC(ZSTD(10))                      -- кодек сжатия
        , col_comment Date COMMENT 'Some comment'               -- комментарий к колонке
    )
    ENGINE = Log;
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.test_fields_without_ttl (
        col_default,
        col_codec,
        col_comment
    )
    SELECT
        number,
        'какой-то текст ' || toString(number),
        toDate(now()) + number
    FROM numbers(60);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.test_fields_without_ttl
''')

А теперь посмотрим как работает TTL для колонкок

In [ ]:


client.command(f'''
    DROP TABLE IF EXISTS {database}.test_fields_with_ttl;
''')


client.command(f'''
    CREATE TABLE {database}.test_fields_with_ttl
    (
          col_default UInt64 DEFAULT 42
        , col_materialized UInt64 MATERIALIZED col_default * 33
        , col_alias UInt64 ALIAS col_default + 1
        , col_codec String CODEC(ZSTD(10))
        , col_comment Date COMMENT 'Some comment'
        , col_ttl UInt64 DEFAULT 10  TTL col_comment + INTERVAL 5 DAY
    )
    ENGINE = MergeTree()
    ORDER BY (col_default);
''')

client.command(f'''
    INSERT INTO {database}.test_fields_with_ttl (
        col_default,
        col_codec,
        col_comment,
        col_ttl
    )
        SELECT
            number,
            'какой-то текст' ||  toString(number),
            toDate(now()) - number,
            rand(1) % 100000000
        FROM numbers(20);
''')

In [ ]:
client.query_df(f'''
    SELECT 
        col_default
      , col_materialized 
      , col_alias 
      , col_codec 
      , col_comment
      , col_ttl
    FROM {database}.test_fields_with_ttl
''')

С помощью следующей команды можно увидеть комментарий к столбцу

In [ ]:
client.query_df(f'''
    DESCRIBE TABLE {database}.test_fields_with_ttl -- здесь можно увидеть комментарий к столбцу
''')


### Самостоятельная работа на расшиненные атрибуты колонок:

1) Создайте таблицу `product_sales`, которая будет хранить информацию о продажах товаров. В таблице необходимо использовать **все указанные выше атрибуты хотя бы по одному разу**.

| Поле          | Тип данных  | Атрибуты                                                              |
|---------------|-------------|------------------------------------------------------------------------|
| `sale_id`     | `UInt64`    | `COMMENT`: "Уникальный идентификатор продажи"                        |
| `price`       | `Float32`   | `DEFAULT`: 0.0                                                        |
| `quantity`    | `UInt8`     | `DEFAULT`: 1                                                          |
| `total`       | `Float32`   | `MATERIALIZED`: `price * quantity`                                   |
| `description` | `String`    | `CODEC`: `ZSTD(1)`                                                    |
| `taxed_total` | `Float32`   | `ALIAS`: `total * 1.2`
2) Вставь пару строк в данную таблицу данные в данную таблицу
3) Выведи данные через колонки и через `*`.
4) Попробуй вставить данные в колонки `total` и `taxed_total`. Посмотри на результат

### Решение(Смотреть только после выполенения)

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.product_sales;
''')


client.command(f'''
    CREATE TABLE {database}.product_sales
    (
        sale_id UInt64 COMMENT 'Уникальный идентификатор продажи',
        price Float32 DEFAULT 0.0,
        quantity UInt8 DEFAULT 1,
        total Float32 MATERIALIZED price * quantity,
        description String CODEC(ZSTD(1)),
        taxed_total Float32 ALIAS total * 1.2
    )
    ENGINE = MergeTree
    ORDER BY sale_id;
''')

client.command(f'''
    INSERT INTO {database}.product_sales (sale_id, price, quantity, description) VALUES
        (1, 10.5, 2, 'Покупка товара А'),
        (2, 5.0, 5, 'Покупка товара Б');
''')

In [ ]:
client.query_df(f'''
    SELECT 
        sale_id
      , price 
      , quantity 
      , total 
      , description
      , taxed_total
    FROM {database}.product_sales
''')

Сравни результаты со следующим запросом. Как ты можешь увидеть не хватает 2х колонок

In [ ]:
client.query_df(f'''
    SELECT 
        *
    FROM {database}.product_sales
''')

Вставлять данные в материализованные и алиас колонки нельзя. Поверь в следующем запросе

In [ ]:
client.command(f'''
    INSERT INTO {database}.product_sales (sale_id, price, quantity, total, taxed_total, description) VALUES
    (3, 7.0, 3, 21.0, 25.2, 'Покупка товара В');
''')
# При выполнении произойдет ОШИБКА!!
# Cannot insert column total, because it is MATERIALIZED column.


# **Атрибуты при создании колонок**

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.nl_lc_tabl;
''')

client.command(f'''
    CREATE TABLE {database}.nl_lc_tabl (
        a Nullable(UInt32),         -- разрешает вставку с пропуском значения.
        b LowCardinality(String),   -- ускоряет работу малокоординальных данных(часто повторяющихся)
        c UInt32
    ) ENGINE = MergeTree 
    ORDER BY tuple(); -- определяет порядок сток по порядку вставки данных
''')

client.command(f'''
    INSERT INTO {database}.nl_lc_tabl VALUES (NULL,'test' ,1); -- null вставится корректно
''')
client.command(f'''
    INSERT INTO {database}.nl_lc_tabl VALUES (1,'test2',NULL); -- null вставится как 0.
''')
client.command(f'''
    INSERT INTO {database}.nl_lc_tabl VALUES (1, NULL, 3); -- null вставляется как пустая строка
''')
client.command(f'''
    INSERT INTO {database}.nl_lc_tabl VALUES (1, 'test2', 4);
''')

In [ ]:
client.query_df(f'''
    SELECT 
        a, 
        b, 
        c 
    FROM {database}.nl_lc_tabl
''')

Сущесвует парамерт который запретит вставлять данные NULL в не Nullable колонки. При которой будут ошибки

In [ ]:

client.command(f'''
    SET input_format_null_as_default = 0; -- параметр отвечающий за вставку пустых значений. Выполняется совместно с командой на вставку
''')
# появится ошибка при вставке
client.command(f'''
    INSERT INTO {database}.nl_lc_tabl VALUES (2, 'sdasda', NULL)
''')


In [ ]:
client.query_df(f'''
    SELECT 
        toTypeName(a), 
        toTypeName(b), 
        toTypeName(c) 
    FROM {database}.nl_lc_tabl
''')

# **Партицирование**

**Партицирование** - это процессобъединения наборов данных по единому критерию, например по месяцу. Это позволяет ускорить процес считывания данных по выбранному критерию при фильтрации.
В ClickHouse существует 4 типа партицирования

## Диапозоном

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_range;
''')

client.command(f'''
    CREATE TABLE {database}.table_range
    (
        id UInt32,
        name String,
        created_at Date
    )
    ENGINE = MergeTree
    PARTITION BY
        CASE
            WHEN id < 10000 THEN 'range_1'
            WHEN id < 20000 THEN 'range_2'
            ELSE 'range_3'
        END
    ORDER BY id;
''')

client.command(f'''
    INSERT INTO {database}.table_range
    SELECT
        number AS id,
        concat('User_', toString(number)) AS name,
        today() AS created_at
    FROM
        numbers(30000)
''')

client.command(f'''
    OPTIMIZE TABLE {database}.table_range FINAL;
''')

In [ ]:
# посмотреть какие у таблицы партиции
client.query_df(f'''
    select 
        _part,
        count(*) 
    FROM 
        {database}.table_range 
    GROUP BY _part 
''')

## Интервалом

Чаще всего в проектах, даже не больших, встречается именно этот вид партицирования

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_interval;
''')

client.command(f'''
    CREATE TABLE {database}.table_interval
    (
        id UInt32,
        amount Float32,
        sale_date Date
    )
    ENGINE = MergeTree
    PARTITION BY toYYYYMM(sale_date)
    ORDER BY id;
''')

client.command(f'''
    INSERT INTO {database}.table_interval
    SELECT
        number AS id,
        rand() % 1000 AS amount,
        today() + (number % 90) AS sale_date
    FROM numbers(1000);
''')

client.command(f'''
    OPTIMIZE TABLE {database}.table_interval FINAL;
''')

In [ ]:
# посмотреть какие у таблицы партиции
client.query_df(f'''
    select 
        _part,
        count(*) 
    FROM 
        {database}.table_interval 
    GROUP BY _part 
''')

## хеш-функцией

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_hash;
''')

client.command(f'''
    CREATE TABLE {database}.table_hash
    (
        user_id UInt64,
        event String
    )
    ENGINE = MergeTree
    PARTITION BY cityHash64(user_id) % 10
    ORDER BY user_id;
''')

client.command(f'''
    INSERT INTO {database}.table_hash
    SELECT
        number AS user_id,
        concat('event_', toString(number))
    FROM numbers(1000);
''')

client.command(f'''
    OPTIMIZE TABLE {database}.table_hash FINAL;
''')

In [ ]:
# посмотреть какие у таблицы партиции
client.query_df(f'''
    select 
        _part,
        count(*) 
    FROM 
        {database}.table_hash 
    GROUP BY _part 
''')

## комбинированое 

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_composiste;
''')

client.command(f'''
    CREATE TABLE {database}.table_composiste
    (
        order_id UInt64,
        customer_id UInt64,
        order_date Date
    )
    ENGINE = MergeTree
    PARTITION BY (toYYYYMM(order_date), customer_id % 10)
    ORDER BY order_id;
''')

client.command(f'''
    INSERT INTO {database}.table_composiste
    SELECT
        number AS order_id,
        number % 100 AS customer_id,
        today() + (number % 90) AS order_date
    FROM numbers(1000);
''')

client.command(f'''
    OPTIMIZE TABLE {database}.table_composiste FINAL;
''')



In [ ]:
# посмотреть какие у таблицы партиции
client.query_df(f'''
    select 
        _part,
        count(*) 
    FROM 
        {database}.table_composiste 
    GROUP BY _part 
''')

### Самостоятельная работа на партиционирование таблиц:
1) Создайте таблицу `customer_orders` с партиционированием по месяцу `PARTITION BY toYYYYMM(order_date)` и первичным ключом сортирвки  `ORDER BY` - `customer_id, order_date`. Поле `total` должно быть `MATERIALIZED`. 

| Поле             | Тип         | Описание                                 |
|------------------|-------------|------------------------------------------|
| `order_id`       | UInt64      | Уникальный идентификатор заказа          |
| `order_date`     | DateTime    | Дата и время заказа                      |
| `customer_id`    | UInt32      | ID клиента                               |
| `product_name`   | String      | Название товара                          |
| `quantity`       | UInt8       | Кол-во единиц                            |
| `price`          | Float32     | Цена за единицу                          |
| `total`          | Float32     | MATERIALIZED: `quantity * price`         |

2) Вставьте данные за несколько разных месяцев по  (можно использовать `now() - INTERVAL X DAY`)
4) Выведи данные на экран с помощью `*` и убедись в какой парции нахаодятся твои данные добавив скрытую колонку `_part`

### Решение(Смотреть только после выполенения)

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.customer_orders;
''')

client.command(f'''
    CREATE TABLE {database}.customer_orders
    (
        order_id UInt64,
        order_date DateTime,
        customer_id UInt32,
        product_name String,
        quantity UInt8,
        price Float32,
        total Float32 MATERIALIZED quantity * price
    )
    ENGINE = MergeTree
    PARTITION BY toYYYYMM(order_date)
    ORDER BY (customer_id);
''')

client.command(f'''
    INSERT INTO {database}.customer_orders (order_id, order_date, customer_id, product_name, quantity, price) VALUES
        (1, now() - INTERVAL 100 DAY, 101, 'Товар A', 2, 150.0),
        (2, now() - INTERVAL 65 DAY, 102, 'Товар B', 1, 300.0),
        (3, now() - INTERVAL 32 DAY, 103, 'Товар C', 3, 200.0),
        (4, now(), 101, 'Товар A', 5, 120.0);
''')

In [ ]:
client.query_df(f'''
    SELECT _part, * FROM {database}.customer_orders 
    WHERE order_date  >= '2025-04-01'
''')

# **Движки таблиц**

**SummingMergeTree** - таблица с группировкой одинаковых записей по ключу сортировки и применением суммы к перечисленным полям 

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.summing_mt;
''')

client.command(f'''
    CREATE TABLE {database}.summing_mt
    (
        id UInt32,
        val UInt32,
        dt datetime,
        example UInt32  -- столбец, не входящий в ключ сортировки и параметры движка
    )
    ENGINE = SummingMergeTree(val) -- сумма будет считаться по полю val, так как оно указано в качестве параметра движка 
    ORDER BY (id)
    PARTITION BY toYYYYMM(dt); -- записи по этому ключу будут группироваться
''')

client.command(f'''
    INSERT INTO {database}.summing_mt
    SELECT 
        number % 2, 
        (number + 1) * 10, 
        now() + number * 60 * 60 * 24,
        (number + 1) * 100 
    from numbers(30);
''')

Запомни что данные сливаются только в рамках партиции

In [ ]:
client.query_df(f'''
    select * from {database}.summing_mt
''')

Давай добавим еще данных и  посмотрим на разницу с предыдущим запросом

In [ ]:
client.command(f'''
    INSERT INTO {database}.summing_mt
    SELECT 
        number % 2, 
        (number + 1) * 10, 
        now() + number * 60 * 60 * 24,
        (number + 1) * 100 
    from numbers(30);
''')

In [ ]:
client.query_df(f'''
    select * from {database}.summing_mt
''')

Как мы видим создался новый блок данных, теперь эти блоки необходими слить друг с другом

In [ ]:
client.command(f'''
    OPTIMIZE TABLE {database}.summing_mt FINAL; -- ручное слияние
''')

In [ ]:
client.query_df(f'''
    select * from {database}.summing_mt
''')

Чтобы увидеть конечный результат и в рамках ключа сортировки оставалось одно значение, то необходимо добавить ключевое слово FINAL в конструкцию FROM. Обрати внимание что данные просуммировались

In [ ]:
client.query_df(f'''
    select * from {database}.summing_mt FINAL
''')

**AggregatingMergeTree** -- это таблица, которая группирует одинаковые записи по ключу сортировки и применяет агрегатные функции к полям

#### Комбинаторы агрегатных функций
https://clickhouse.com/docs/ru/sql-reference/aggregate-functions/combinators

In [ ]:
client.query_df('''
    with t1 as (
    select number, 
            number * 10 as colsum,
            number % 3 as coldist
    from numbers(10)
    )
    select 
        sumIf(colsum, number % 2 == 0),
        countDistinct(coldist),
        countDistinctIf(coldist, coldist % 2 = 0)
    from t1
''')

### Самостоятельная работа на комбинаторы:

1) Выполни запрос запрос, чтобы создать тестовую таблицу
2) Напиши 3 запроса отвечающие на следующие вопросы:
- Сколько уникальных пользователей заходили с **мобильных устройств**
- Среднее время просмотра только на странице `home`
- Для каждого устройства — количество уникальных пользователей
- Собери список страниц, на которые заходил пользователь в индефикатором `2`, только с десктопа (`web`)

**Подсказака**:
В запросах нужно исполльзовать следующие компненты: `groupArray`, `If`, `avg`, `uniq`

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.page_views;
''')

client.command(f'''
    CREATE TABLE {database}.page_views
    (
        user_id UInt32,
        page String,
        device String,
        view_time UInt8
    )
    ENGINE = MergeTree
    ORDER BY user_id;
''')

client.command(f'''
    INSERT INTO {database}.page_views (user_id, page, device, view_time) VALUES
        (1, 'home', 'mobile', 10),
        (1, 'catalog', 'mobile', 15),
        (2, 'home', 'web', 8),
        (2, 'checkout', 'web', 20),
        (2, 'home', 'mobile', 5),
        (4, 'catalog', 'web', 12),
        (5, 'checkout', 'mobile', 7),
        (6, 'home', 'web', 9),
        (7, 'home', 'mobile', 11),
        (8, 'catalog', 'web', 13);
''')

### Решение(Смотреть только после выполенения)

2.1 Сколько уникальных пользователей заходили с **мобильных устройств**

In [ ]:
client.query_df(f'''
    SELECT 
        uniqIf(user_id, device = 'mobile') AS mobile_users 
    FROM {database}.page_views
''')

2.2 Среднее время просмотра только на странице `home`

In [ ]:
client.query_df(f'''
    SELECT 
        avgIf(view_time, page = 'home') AS avg_home_time 
    FROM {database}.page_views
''')

2.3 Для каждого устройства — количество уникальных пользователей

In [ ]:
client.query_df(f'''
    SELECT 
        device, 
        uniq(user_id) 
    FROM {database}.page_views
    GROUP BY device
''')

2.4 Собери список страниц, на которые заходил пользователь в индефикатором `2`, только с десктопа (`web`)

In [ ]:
client.query_df(f'''
    SELECT
        user_id,
        groupArrayIf(page, device = 'web') AS web_pages_visited
    FROM {database}.page_views
    WHERE user_id = 2
    GROUP BY user_id    
''')

#### Агрегаторные типы данных

Сущесвует 2 агригаторных типа данных:
- SimpleAggregateFunction - предназначет для хранения простых агрегатов, которое хранит конечно состояние
- AggregateFunction - сложные агрегаты, которая хранит состояние всех добавленных значений

Разберем пример с **SimpleAggregateFunction**

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.simple;
''')

client.command(f'''
    CREATE TABLE {database}.simple (
      id UInt64, 
      val_sum SimpleAggregateFunction(sum, UInt64), -- предусмотрен для хранения простых агрегатов(хранит конечное состояние)
      val_max SimpleAggregateFunction(max, UInt32)
    ) 
    ENGINE=AggregatingMergeTree 
    ORDER BY id;
''')

client.command(f'''
    INSERT INTO {database}.simple SELECT  1, number, number from numbers(10);
''')
client.command(f'''
    INSERT INTO {database}.simple SELECT  2, sum(number), max(number) from numbers(5);
''')
client.command(f'''
    INSERT INTO {database}.simple SELECT  1, number, number from numbers(8);
''')

Выведим запрос и увидем, что движок агрегирует данные в блоке данных при вставке. В итоге мы увидем 3 строки

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.simple
''')

Но нам же нужно для ID=1 получить 1 строку. Можно дождаться следующего слияния, а можно выполнитьследующий запрос, которое сделает **логическое** слияние и выдаст конечный результат

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.simple FINAL
''')

На больших таблицах намного эффективнее получить тот же результат следущим запросом

In [ ]:
# лучше делать так

client.query_df(f'''
    SELECT 
        id, 
        sum(val_sum),
        max(val_max) 
    FROM {database}.simple
    GROUP BY id
''')

Разберем пример с **AggregateFunction**

Здесь немного все посложнее так как нужно использовать специальные комбинаторы

Комбинаторы агрегаторных типов данных:
* SimpleState — возвращает результат агрегирующей функции типа SimpleAggregateFunction.
* State — возвращает промежуточное состояние типа AggregateFunction, используется при вставке.
* Merge — берёт множество состояний, объединяет их и возвращает результат полной агрегации данных.
* MergeState — выполняет слияние промежуточных состояний агрегации, возвращает промежуточное состояние агрегации.

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.aggr_func_tbl;
''')

client.command(f'''
    CREATE TABLE {database}.aggr_func_tbl
    (
        id UInt64,
        val_uniq AggregateFunction(uniq, UInt64),         -- Хранит в себе промежуточное состояние данных
        val_any AggregateFunction(anyIf, String, UInt8),
        val_quant AggregateFunction(quantiles(0.5, 0.9), UInt64)
    ) ENGINE=AggregatingMergeTree 
    ORDER BY id;
''')

client.command(f'''
    INSERT INTO {database}.aggr_func_tbl
    SELECT 
        1, 
        uniqState(toUInt64(rnd)),                 -- кол-во уникальных значений
        anyIfState(toString(rnd),rnd%2=0),
        quantilesState(0.5, 0.9)(toUInt64(rnd)) 
    FROM
        (SELECT arrayJoin(arrayMap(i -> i * 10, range(10))) as rnd);
''')

In [ ]:
# вставь эту строку в бобра иначе не выполнится
client.query_df(f'''
    SELECT * FROM {database}.aggr_func_tbl FORMAT Vertical -- промежуточные значения в бинарном виде
''')

In [ ]:
client.query_df(f'''
       SELECT uniqMerge(val_uniq), 
              quantilesMerge(0.5, 0.9)(val_quant), 
              anyIfMerge(val_any) 
       FROM {database}.aggr_func_tbl
''')
 

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.simple_aggregating_table
''')
 

Рассмотрим еще один пример. Обращаю внимание на нахождение среднего.

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.aggregating_table;
''')

client.command(f'''
    CREATE TABLE {database}.aggregating_table
    (
        id UInt32,
        val_count AggregateFunction(count, UInt64),
        val_avg   AggregateFunction(avg, Float64),
        val_groupArray AggregateFunction(groupArray, UInt64)
    )
    ENGINE = AggregatingMergeTree
    ORDER BY (id);
''')

client.command(f'''
    INSERT INTO {database}.aggregating_table
    with t1 as (
    SELECT number % 4 as id,
        (number + 1) * 1 col1,
        (number + 1) * 1 col2,
        (number + 1) * 1 col3
    from numbers(10)
    )
    select
        id, 
        countState(col1),
        avgState(toFloat64(col2)), -- Обратите внимание на toFloat64. ClickHouse не может автоматически привести 
                                    -- avgState(UInt64) → avgState(Float64), даже если кажется, 
                                    -- что avg всё равно возвращает float.
        groupArrayState(col3)
    from t1
    group by id; 
''')

In [ ]:
# напоминаю что такой результат выдаст ошибку только так FORMAT Vertical

client.query_df(f'''
    SELECT  *  FROM {database}.aggregating_table
''')

In [ ]:
client.query_df(f'''
    SELECT
        id,
        countMerge(val_count)        AS count_val,
        avgMerge(val_avg)            AS avg_val,
        groupArrayMerge(val_groupArray) AS grouped_vals
    FROM {database}.aggregating_table
    group by id
''')

# получается, что вы сохранили сначала avg(10, 20, 30), а затем avg(1, 2, 3). 
# Итоговый результат, который вы получите, будет avg(1, 2, 3, 10, 20, 30). 

### Самостоятельная работа на агригационные движки:

1) Давай возьмем запро, который мы рассматривали при изучени SummingMergeTree(он в следующей ячеке)
2) Измени его таким образом чтобы агрегировалась **Сумма** по `val` и **максимальное заначение** по дате.
3) Вравни значения движка SummingMergeTree и AggregatingMergeTree

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.summing_mt_dz;
''')

client.command(f'''
    CREATE TABLE {database}.summing_mt_dz
    (
        id UInt32,
        val UInt32,
        dt datetime,
        example UInt32  -- столбец, не входящий в ключ сортировки и параметры движка
    )
    ENGINE = SummingMergeTree(val) -- сумма будет считаться по полю val, так как оно указано в качестве параметра движка 
    ORDER BY (id)
    PARTITION BY toYYYYMM(dt); -- записи по этому ключу будут группироваться
''')

client.command(f'''
    INSERT INTO {database}.summing_mt_dz
    SELECT 
        number % 2, 
        (number + 1) * 10, 
        now() + number * 60 * 60 * 24,
        (number + 1) * 100 
    from numbers(30);
''')

In [97]:
client.query_df(f'''
    SELECT * FROM {database}.summing_mt_dz
''')

,id,val,dt,example
0,0,90,2025-06-25 20:08:44+03:00,100
1,1,120,2025-06-26 20:08:44+03:00,200
2,0,2160,2025-07-01 20:08:44+03:00,700
3,1,2280,2025-07-02 20:08:44+03:00,800


### Решение(Смотреть только после выполенения)

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.summing_mt_dz;
''')

client.command(f'''
    CREATE TABLE {database}.summing_mt_dz
    (
        id UInt32,
        val SimpleAggregateFunction(sum, UInt64),
        dt SimpleAggregateFunction(max, DateTime),
        example UInt32 
    )
    ENGINE=AggregatingMergeTree 
    ORDER BY id
    PARTITION BY toYYYYMM(dt);
''')

client.command(f'''
    INSERT INTO {database}.summing_mt_dz
    SELECT 
        number % 2, 
        (number + 1) * 10, 
        now() + number * 60 * 60 * 24,
        (number + 1) * 100 
    from numbers(30);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.summing_mt_dz
''')

 **ReplacingMergeTree** -- удаляет дублирующиеся записи с одинаковым значением ключа сортировки.

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.replacing_merge_tree;
''')

client.command(f'''
    CREATE TABLE {database}.replacing_merge_tree
    (
        id UInt32,
        dt date,
        val UInt32
    )
    ENGINE = ReplacingMergeTree(id)
    ORDER BY (id, dt);
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.replacing_merge_tree
    SELECT 1, 
        now()::date,
        (number + 1) * 400
    FROM numbers(1); 
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.replacing_merge_tree
''')

In [ ]:
client.command(f'''
    OPTIMIZE TABLE {database}.replacing_merge_tree;
''')

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.replacing_merge_tree_with_version;
''')

client.command(f'''
    CREATE TABLE {database}.replacing_merge_tree_with_version
    (
        id UInt32,
        dt date,
        val UInt32
    )
    ENGINE = ReplacingMergeTree(dt) -- dt может быть и числовой колонкой
    ORDER BY (id)
    PARTITION BY toYYYYMM(dt);
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.replacing_merge_tree_with_version
    SELECT 
        1, 
        now()::date + number - 15,
        (number + 1) * 1000
    FROM numbers(10);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.replacing_merge_tree_with_version
''')


In [ ]:
client.command(f'''
    OPTIMIZE TABLE {database}.replacing_merge_tree_with_version FINAL;
''')

**CollapsingMergeTree** -- Удаляет дубликаты по ключу сортировки в зависимости от флага

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.Books;
''')

client.command(f'''
    CREATE TABLE {database}.Books
    (
        ID UInt64,
        Page UInt8,
        Sign Int8 -- имеет только 2 значения "1" и "-1"
    )
    ENGINE = CollapsingMergeTree(Sign)
    ORDER BY ID;
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.Books values (1, 1, 1);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.Books
''')

In [ ]:
client.command(f'''
    INSERT INTO {database}.Books values (1, 1, -1),(1, 2, 1);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.Books
''')

In [ ]:

client.command(f'''
   OPTIMIZE TABLE {database}.Books;
''')

# в рамках ключа будет оставаться всегда последняя добавленая строка с "+1". 
# все строки с "-1" будут удалены 

**Log** -- для небольших таблиц. Каждая колока отдельный файл

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.el;
''')

client.command(f'''
    CREATE TABLE {database}.el
    (
        id UInt32,
        dt date
    )
    ENGINE = Log
''')

client.command(f'''
    INSERT INTO {database}.el
    select 
    number,
    now()::date + number,
    from numbers(10);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.el
''')

**File** -- позволяет записывать данные в формате файла

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.ef;
''')

client.command(f'''
    CREATE TABLE {database}.ef
    (
        id UInt32,
        dt date
    )
    ENGINE = File(CSV);
''')

client.command(f'''
    INSERT INTO {database}.ef
    SELECT 
      number,
      now()::date + number
    FROM numbers(10);
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.ef
''')

**Buffer** -- для укорения вставки в таблицы. Данные записываются в ОП далее сливаются в другую таблицу

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.eb;
''')

client.command(f'''
    DROP TABLE IF EXISTS {database}.ebt;
''')

client.command(f'''
    CREATE TABLE {database}.eb
    (
        id UInt32,
        dt date
    )
    ENGINE = Buffer({database}, -- имя БД
                    ebt,     -- имя таблицы для слива данных
                    16,      -- параллелизм (рекомендация 16)
                    30,      -- минимальное время слияния
                    60,      -- минимальное время слияния
                    5,       -- минимальное кол-во строк для слияния
                    10,      -- максимальное кол-во строк для слияния
                    10000,   -- минимальное кол-во байт для слияния
                    10000    -- максимальное кол-во байт для слияния
                    );
''')



In [ ]:
client.command(f'''
    INSERT INTO {database}.eb
    select 
        number,
        now()::date + number
    from numbers(1);
''')

In [ ]:
# будет ошибка
client.query_df(f'''
    SELECT * FROM {database}.eb
''')

In [ ]:
client.command(f'''
    CREATE TABLE {database}.ebt
        (
        id UInt32,
        dt date
    )
    ENGINE = Log;
''')

In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.ebt
''')

In [ ]:
client.command(f'''
    OPTIMIZE TABLE eb;
''')

**Memory** --данные хранятся только в оперативной памяти. При перезапуске CH данные будут утеряны

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.em;
''')

client.command(f'''
    CREATE TABLE {database}.em
    (
        id UInt32,
        dt date
    )
    ENGINE = Memory;
''')

client.command(f'''
    INSERT INTO {database}.em
    SELECT 
        number,
        now()::date + number
    FROM numbers(100);
''')





In [ ]:
client.query_df(f'''
    SELECT * FROM {database}.em
''')

**Set** -- Движок предназначен для использования в правой части оператора IN. Не хранятся дублирующие значения

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.es;
''')

client.command(f'''
    CREATE TABLE {database}.es
    (
        id UInt32
    )
    ENGINE = Set
    SETTINGS persistent = 1; -- данные будут считываться из ОП
''')

client.command(f'''
    INSERT INTO {database}.es SELECT number from numbers(30);
''')

In [ ]:
# читать данные из такой таблицы нельзя, только IN только хардкор
client.query_df(f'''
    SELECT * FROM {database}.es
''')

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.est;
''')

client.command(f'''
    CREATE TABLE {database}.est
    (
        id UInt32
    )
    ENGINE = MergeTree
    ORDER BY (id);
''')

client.command(f'''
    INSERT INTO {database}.est SELECT number from numbers(300);
''')

In [ ]:
client.query_df(f'''
    SELECT *
    FROM {database}.est
    WHERE id in {database}.es
''')

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.es2;
''')

client.command(f'''
    DROP TABLE IF EXISTS {database}.est2;
''')

client.command(f'''
    CREATE TABLE {database}.es2
    (
        id UInt32,
        id2 UInt32
    )
    ENGINE = Set
    SETTINGS persistent = 1;
''')

client.command(f'''
    CREATE TABLE {database}.est2
    (
        id UInt32,
        id2 UInt32
    )
    ENGINE = MergeTree
    ORDER BY (id);
''')

client.command(f'''
    INSERT INTO {database}.es2 SELECT number, number from numbers(30);
''')

client.command(f'''
    INSERT INTO {database}.est2 SELECT number, number from numbers(300);
''')

In [ ]:
client.query_df(f'''
    SELECT *
    FROM {database}.est2
    WHERE (id, id2) in {database}.es2
''')

**GenerateRandom** -- предназначен для генерации данных в СН для дальнейших тестов

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.eg;
''')

client.command(f'''
    CREATE TABLE {database}.eg
    (
        id UInt32, 
        val String,
        dt date,
        a Float32,
        b UUID,
        c Bool,
        d IPv6,
        e IPv4,
        g Array(UInt32)
    )
    ENGINE = GenerateRandom;
''')

In [ ]:
client.query_df(f'''
    select * 
    from {database}.eg
    limit 10
''')

**PostgreSQL** -- для работы с таблицами БД PSQL

In [2]:
import psycopg2 as ps
import pandas as pd
import os

schema = 'shustikov' # В расках схемы задайте свою фамилию

conn = ps.connect(host="postgres_source", 
                  port = 5432, 
                  database="dev", 
                  user=os.getenv("POSTGRES_USER"), 
                  password=os.getenv("POSTGRES_PASSWORD"))

cursor = conn.cursor()

cursor.execute(f'''
    CREATE SCHEMA IF NOT EXISTS {schema};
    ''')
    
cursor.execute(f'''
    DROP TABLE IF EXISTS {schema}.departments CASCADE;
''')

cursor.execute(f'''
    CREATE TABLE {schema}.departments (
        dept_id SERIAL PRIMARY KEY,
        dept_name VARCHAR(50),
        location VARCHAR(50)
    )
''')

cursor.execute(f'''
    INSERT INTO {schema}.departments (dept_name, location) VALUES
    ('HR', 'Москва'),
    ('IT', 'Санкт-Петербург'),
    ('Finance', 'Москва'),
    ('DE', 'Краснодар')
''')

conn.commit()

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.postgresql_table;
''')

client.command(f'''
    CREATE TABLE {database}.postgresql_table
    (
        dept_id Int32,
        dept_name String,
        location String
    )
    ENGINE = PostgreSQL(
                        'postgres_source:5432', -- сервер, порт
                        'dev',              -- БД 
                        'departments',      -- таблица
                        '{os.getenv("POSTGRES_PASSWORD")}',         -- логин
                        '{os.getenv("POSTGRES_PASSWORD")}',         -- пароль
                        'shustikov'         -- имя схемы
                        );
''')

In [ ]:
client.query_df(f'''
     select * from {database}.postgresql_table
''')

### Самостоятельная работа на движок PSQL

1) Выполни подключение к любой своей таблице и выполни селект запрос

**Kafka** - с помощью данного движка можно, как **отправлять** данные в кафку так и **получать** данные из кафки

**Отправляем** данные в кафку

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.kafka_out_message;
''')

client.command(f'''
    CREATE TABLE {database}.kafka_out_message
    (
        id UInt32
    )
    ENGINE = Kafka
    SETTINGS
        kafka_broker_list = 'kafka:29093',
        kafka_topic_list = '{database}.clickhouse.topic', -- создай такой же топик
        kafka_group_name = 'clickhouse_consumer_group',
        kafka_format = 'JSONEachRow';
''')   

После вставки данных проверь свой топик на сообщения

In [ ]:
client.command(f'''
    INSERT INTO {database}.kafka_out_message
    SELECT number FROM numbers(30);
''')

**Читаем** данные из кафки

In [48]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.kafka_input_stage;
''')

client.command(f'''
    CREATE TABLE {database}.kafka_input_stage
    (
        json_kafka String                               -- задается только 1 строка с полным JSON-ом из кафки
    )
    ENGINE = Kafka
    SETTINGS
        kafka_broker_list = 'kafka:29093',
        kafka_topic_list = 'source.public.order_events',
        kafka_group_name = 'clickhouse_consumer_group',
        kafka_format = 'JSONAsString';                  -- достаточно часто истользуется именно этот формат
''')

Давайте посмотрим, храниться в самой таблице

In [ ]:
# Читать из такой таблицы нельзя, при выполенении такого запроса будет ошибка
client.query_df(f'''
     SELECT * FROM {database}.kafka_input_stage
''')

Следующим способом можно посмотреть приходят ли данные или нет

In [ ]:
client.command(f'''
    SET stream_like_engine_allow_direct_select = 1
''')

client.query_df(f'''
     SELECT * FROM {database}.kafka_input_stage
''')

Теперь посмотрим как правильно хранить данные в ClickHouse

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.table_stage_from_kafka;
''')

client.command(f'''
   create table {database}.table_stage_from_kafka 
   (
      json_kafka String
   )
   Engine = Log
''')

client.command(f'''
    DROP VIEW IF EXISTS {database}.kafka_input_stage_mv;
''')

client.command(f'''
   CREATE MATERIALIZED VIEW {database}.kafka_input_stage_mv TO {database}.table_stage_from_kafka AS 
      SELECT
         json_kafka
      FROM {database}.kafka_input_stage;
''')

Как прочитать данные из строки в формате JSON

In [ ]:
client.query_df(f'''
     SELECT  
          JSONExtractInt(JSONExtractString(json_kafka ,'before'), 'id') as before_id,               -- JSONExtract<тип данных>(<JSON строка>, <ключ>)
          JSONExtractInt(JSONExtractString(json_kafka ,'before'), 'order_id') as before_order_id,
          JSONExtractString(JSONExtractString(json_kafka ,'before'), 'status') as before_status,
          JSONExtractUInt(JSONExtractString(json_kafka ,'before'), 'ts') as before_ts,
          JSONExtractInt(JSONExtractString(json_kafka ,'after'), 'id') as after_id,
          JSONExtractInt(JSONExtractString(json_kafka ,'after'), 'order_id') as after_order_id,
          JSONExtractString(JSONExtractString(json_kafka ,'after'), 'status') as after_status,
          JSONExtractUInt(JSONExtractString(json_kafka ,'after'), 'ts') as after_ts,
          JSONExtractString(json_kafka ,'op') as op,
          toDateTime(JSONExtractInt(json_kafka ,'ts_ms') / 1000) as dt                              -- Перевод времени из timestamp в читаемый вид
     FROM {database}.table_stage_from_kafka
''')



### Самостоятельная работа на движок Kafka

Ты уже много чего прошел, пришло время давать серьезные задачи, как из реально дают в компаниях.(Это реальный кейс с работы)

К тебе приходит тимлид и говорит: Аналитики из команды продаж, просят предоставить им данные от внутреннего сервиса отвечающего за продажи определенных вещей. Они уже договорили с DBA(администратор баз данных) отвечающего за поддержание БД данного сервиса и он настроил топик в кролике(Rabbitmq, тоже самое что и кафка только в профиль), который находится в их структуре. Тебе необходимо положить эти данные в GreenPlum. Ты знаешь, что у вас в команде есть ClikHouse, Kafka, Spark, GreenPlum, но из Spark нельзя достучаться до кролика, так как он находится в другой сети. Spark может читать стримовые данные только из локальной кафки. Доступ есть у Клика, так как та самая комадна кладёт туда свои данные для анализа.

Твоя задача: взять данные из кролика(в роли кролика у нас выступает кафка с топиком **source.public.dbz_heartbeat**) и загнать данные в другой топик с именем **<фамилия>.source.data_from_rabit**. От туда твой мнимый товарисчь, мнимо данные заберёт и положит в мнимый GreenPlum

### Решение(Смотреть только после выполенения)

In [ ]:
client.command(f'''
    DROP TABLE IF EXISTS {database}.data_from_rabbit;
''')

client.command(f'''
   create table {database}.data_from_rabbit 
      (
         json_kafka String
      )
   Engine = Kafka
   SETTINGS
        kafka_broker_list = 'kafka:29093',
        kafka_topic_list = 'source.public.dbz_heartbeat',
        kafka_group_name = 'clickhouse_consumer_group',
        kafka_format = 'JSONAsString'; 
''')

client.command(f'''
    DROP TABLE IF EXISTS {database}.data_rabbit_to_kafka;
''')

client.command(f'''
   create table {database}.data_rabbit_to_kafka 
   (
      before String,
      after String
   )
   Engine = Kafka
   SETTINGS
        kafka_broker_list = 'kafka:29093',
        kafka_topic_list = 'shustikov.source.data_from_rabit',
        kafka_group_name = 'clickhouse_consumer_group',
        kafka_format = 'JSONEachRow';
''')

client.command(f'''
    DROP VIEW IF EXISTS {database}.data_from_rabbit_mv;
''')

client.command(f'''
   CREATE MATERIALIZED VIEW {database}.data_from_rabbit_mv TO {database}.data_rabbit_to_kafka AS 
      SELECT 
         simpleJSONExtractRaw(json_kafka, 'before') as before,
         simpleJSONExtractRaw(json_kafka, 'after') as after
         -- для примера тут приведено всего 2 строки, но надеюсь принцип вы поняли
      from {database}.data_from_rabbit ;
''')

**Распределяем таблицу по шардам**

Если у вас будет распределенный ClickHouse, то данный сценарий у вас будет постояно

In [132]:
#query_df -- для вывода на экран

client.command(f'''
    CREATE TABLE {database}.events ON CLUSTER 'company_cluster' (   -- имя кластера можно задавать по имени и с помощью макроса '{{cluster}}'
        time DateTime,
        uid  Int64,
        type LowCardinality(String)
    )
    ENGINE = ReplicatedMergeTree('/clickhouse/tables/{{cluster}}/{{shard}}/events', '{{replica}}')
    PARTITION BY toDate(time)
    ORDER BY (uid);
''')

client.command(f'''
    CREATE TABLE {database}.events_distr ON CLUSTER 'company_cluster' AS {database}.events
    ENGINE = Distributed('company_cluster', {database}, events, uid);
''')

client.command(f'''
    INSERT INTO {database}.events_distr VALUES
        ('2020-01-01 10:00:00', 100, 'view'),
        ('2020-01-01 10:05:00', 101, 'view'),
        ('2020-01-01 11:00:00', 100, 'contact'),
        ('2020-01-01 12:10:00', 101, 'view'),
        ('2020-01-02 08:10:00', 100, 'view'),
        ('2020-01-03 13:00:00', 103, 'view');
''')

In [ ]:
client.query_df(f'''
    select * from {database}.events_distr 
''')

In [ ]:
client.query_df(f'''
    select * from {database}.events
''')

# **JOIN**

Ошибка распредленного джойна

In [ ]:
client.command(f'''
    drop table {database}.tabl_join_local_1 on CLUSTER '{{cluster}}';
''')

client.command(f'''
    CREATE TABLE {database}.tabl_join_local_1 on CLUSTER '{{cluster}}'
    (
      id1 UInt32,
      id2 UInt32 
    )
    engine = MergeTree
    order by (id1);
''')

client.command(f'''
    drop table {database}.tabl_join_1 on CLUSTER '{{cluster}}';
''')

client.command(f'''
    CREATE TABLE {database}.tabl_join_1 ON CLUSTER '{{cluster}}' AS {database}.tabl_join_local_1
    ENGINE = Distributed('company_cluster', {database}, tabl_join_local_1, id1);
''')

client.command(f'''
    INSERT INTO {database}.tabl_join_1 values (1, 10),(2, 11),(3, 12),(4, 13),(5, 14),(6, 15),(7, 16),(8, 17),(9, 18),(0, 29)
''')

In [ ]:
client.command(f'''
    drop table {database}.tabl_join_local_2 on CLUSTER '{{cluster}}';
''')

client.command(f'''
    CREATE TABLE {database}.tabl_join_local_2 on CLUSTER '{{cluster}}'
    (
      id1 UInt32,
      id2 UInt32 
    )
    engine = MergeTree
    order by (id1);
''')

client.command(f'''
    drop table {database}.tabl_join_2 on CLUSTER '{{cluster}}';
''')

client.command(f'''
    CREATE TABLE {database}.tabl_join_2 ON CLUSTER '{{cluster}}' AS {database}.tabl_join_local_2
    ENGINE = Distributed('company_cluster', {database}, tabl_join_local_2, id2);                                              -- изменен парамент распределения по шардам
''')

client.command(f'''
    INSERT INTO {database}.tabl_join_2 values (1, 10),(2, 11),(3, 12),(4, 13),(5, 14),(6, 15),(7, 16),(8, 17),(9, 19),(0, 29)
''')                                                                                                                          #(9, 19) попадет на один и тот же шард с (9, 18)

Посмотрим какие данные у нас на этой шарде, поменяй потом на tabl_join_local_2. Распределение данных по шардам основывается на остатке от деления

In [ ]:

client.query_df(f'''
    select *
    from {database}.tabl_join_local_2
''')

Попробуем сделать JOIN таблиц, но.....

In [ ]:
# Получишь ошибку, так как кликхаус из коробки не зарешает распределённо джойнить
client.query_df(f'''
    select *
    from {database}.tabl_join_1 as t1  
      JOIN {database}.tabl_join_2 as t2
        ON t1.id1 = t2.id1
''')

GLOBAL - перекачивает данные из всех шардов на координатор и на данной тачке джойнит все на нём, естественно повышая нагрузку на этоот координатор

In [ ]:
client.query_df(f'''
    select *
    from {database}.tabl_join_1 as t1  
      GLOBAL JOIN {database}.tabl_join_2 as t2
        ON t1.id1 = t2.id1
''')

Настройка distributed_product_mode = 'local' разрешает джойнить таблицы локально на каждой шарде. Это может привести к коализиям если не джойнить таблицы по ключу распредления, как в следующем примере

In [ ]:
client.command('''
    SET distributed_product_mode = 'local'  -- по умолчанию deny
''')

client.query_df(f'''
    select *
    from {database}.tabl_join_1 as t1  
      JOIN {database}.tabl_join_2 as t2
        ON t1.id1 = t2.id1
''')

**ASOF JOIN** -- приближенное значение по условию неравентсва

In [ ]:
client.query_df('''
  SELECT 
      number AS k, 
      toDateTime('2020-10-10 10:30:00') + number * 100 as ts, 
      number * 10 AS a
  FROM system.numbers LIMIT 5
''')



In [ ]:
client.query_df('''
      SELECT number AS k, 
          toDateTime('2020-10-10 10:00:00') + number * 100 + 3 as ts, 
          number * 100 AS b
      FROM system.numbers
      LIMIT 5
    UNION ALL
      SELECT number AS k, 
          toDateTime('2020-10-10 11:00:00') + number * 100 + 3 as ts, 
          number * 1000 AS b
      FROM system.numbers
      LIMIT 5
    UNION ALL
      SELECT number AS k,
          toDateTime('2020-10-10 12:00:00') + number * 100 + 3 as ts,
          number * 10000 AS b
      FROM system.numbers
      LIMIT 5
''')
 

In [ ]:
client.query_df('''
    SELECT T_A.k, T_A.ts,  T_B.ts, T_A.a, T_B.b
    FROM
        (
            SELECT number AS k, 
            toDateTime('2020-10-10 10:30:00') + number * 100 as ts, 
            number * 10 AS a
            FROM system.numbers
            LIMIT 5
        ) T_A
    ASOF JOIN
        (
            SELECT number AS k, 
            toDateTime('2020-10-10 10:00:00') + number * 100 + 3 as ts, 
            number * 100 AS b
            FROM system.numbers
            LIMIT 5
            UNION ALL
            SELECT number AS k, 
            toDateTime('2020-10-10 11:00:00') + number * 100 + 3 as ts, 
            number * 1000 AS b
            FROM system.numbers
            LIMIT 5
            UNION ALL
            SELECT number AS k,
            toDateTime('2020-10-10 12:00:00') + number * 100 + 3 as ts,
            number * 10000 AS b
            FROM system.numbers
            LIMIT 5
        ) T_B ON
        T_A.k = T_B.k and
        T_A.ts < T_B.ts        
    ORDER BY T_A.k
        SETTINGS join_use_nulls = 0
''')




### Решение(Смотреть только после выполенения)

Реши задачу с собесендования

Дано:
Представим, что у нас есть интернет магазин, из которого мы собираем данные.
1) данные по логинам пользователей
- данные летят в нашу таблицу в режиме реального времени, каждый раз, когда пользователь логинится на сайт
- таблица: user_login
- поля:
- user_id Uint64
- login_time DateTime
- объем данных большой

2.данные по акциям
- данные летят в нашу таблицу в режиме реального времени, каждый раз, когда создается новая акция
- таблица event
- поля:
- event_name String
- start_time DateTime
- end_time DateTime
- объем данных маленький

Нужно посчитать кол-во логинов пользователя во время действия акйций за выбранный промежуток времени

Удачи!